# Semana 7 do Aceleradev DS Codenation

### Professor: Kazuki Yokoyama | Tema: Feature Engineering

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## O que é Feature Engeeniring?

Todo modelo de aprendizado de máquinas é alimentado com dados, que precisam ser trabalhados para que o modelo aprenda adequadamente e retorne boa predições. Isso é *engenharia de features*! muitas vezes os dados estão fora de escala, ou um dado categórico, se transformado em categoria numérica, pode enriquecer os dados de treino. Essa habilidade do DS é muito importante no processo, pois os dados não vêm *bonitinhos* prontos para serem consumidos, eles precisam ser *engenheirados* primeiro, e grande parte do tempo gasto na resolução de um problema com ML, é nessa etapa. Diante disso, a qualidade dela é diretamente proporcional a do modelo preditivo

**Fixando uma seed**

In [2]:
np.random.seed(1000)

**Criação de DataFrame Aleatório com variáveis categóricas e numéricas**

In [131]:
linhas = 100

In [168]:
altura = np.random.normal(loc=1.7, scale=.2, size=linhas).round(3)
ponto = np.random.normal(loc=7, scale=1,size=linhas).round(3)
cursos = ['mat', 'fis', 'bio']
curso = np.random.choice(a=cursos, size=linhas)

In [169]:
df = pd.DataFrame({'altura': altura,
                  'ponto': ponto,
                  'curso': curso})
df

,altura,ponto,curso
0,1.668,8.550,mat
1,1.584,7.748,bio
2,1.689,6.869,mat
3,1.826,6.446,bio
4,1.300,6.922,mat
...,...,...,...
95,1.854,7.872,fis
96,1.935,7.342,mat
97,1.824,6.814,bio
98,1.473,7.836,fis


`df` é uma tabela que contém três variáveis, duas **numéricas** e uma **categórica**

## Começando a Engenharia de Features

### Variáveis Categóricas

**One-hot Encoding:** As $n$ categorias da variável categórica serão usadas para criação de novas colunas na sua tabela. Cada categoria representará uma coluna. Como uma tupla consegue ter apenas uma categoria, a coluna correspondente à categoria terá o valor 1 (Hot) preenchido, enquanto as demais terão 0 (Cold) preenchido. Perceba que teremos novas variáveis **binarias** em nosso `df`

Para fazer isso, usaremos uma classe do sklearn.preprocessing, o **OneHotEncoder**

```python
sklearn.preprocessing.OneHotEncoder(sparse=False, dtype=np.uint8)
```

**Nota:** caso deixe `sparse=True`, vc terá uma matriz `sparse matrix`. Ela economiza mais memória, mas não mostra a matriz diretamente. Para revelar então

```python
sparse_matrix.toarray()
```

In [170]:
from sklearn.preprocessing import OneHotEncoder

In [171]:
one_hot_enc = OneHotEncoder(sparse=False, dtype=np.uint8)

In [172]:
course_enc = one_hot_enc.fit_transform(df[['curso']])  # lembre de colocar [[]] pro atributo entender que é um (n x 1)

In [173]:
course_enc.shape

(100, 3)

In [174]:
course_enc[:5]

array([[0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1]], dtype=uint8)

In [175]:
one_hot_enc.categories_

[array(['bio', 'fis', 'mat'], dtype=object)]

In [176]:
cat_columns = one_hot_enc.categories_[0]

Podemos concatenar essas informações ao `df`

In [177]:
df_dois = df.join(pd.DataFrame(course_enc, columns=cat_columns))
df_dois.head()

,altura,ponto,curso,bio,fis,mat
0,1.668,8.550,mat,0,0,1
1,1.584,7.748,bio,1,0,0
2,1.689,6.869,mat,0,0,1
3,1.826,6.446,bio,1,0,0
4,1.300,6.922,mat,0,0,1


**Binarização:** Processo que discretiza uma variável numérica contínua, tendo um certo *limite* como parâmetro. O valor **acima do limite** recebe 1, já **abaixo do limite** recebe 0

Para fazer isso, usaremos uma classe do sklearn.preprocessing, o **Binarizer**
```python
sklearn.preprocessing.Binarizer(threshold=float)
```

In [178]:
from sklearn.preprocessing import Binarizer

O limite para `altura` será de 1.8

In [179]:
binarizer = Binarizer(threshold=1.8)

In [180]:
binary = binarizer.fit_transform(df[['altura']])

In [181]:
binary.flatten()

array([0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0.,
       0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1., 1.,
       1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1.,
       1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0.])

In [183]:
df_tres = df_dois.join(pd.Series(binary.flatten(), name='alto'))
df_tres.head()

,altura,ponto,curso,bio,fis,mat,alto
0,1.668,8.550,mat,0,0,1,0.0
1,1.584,7.748,bio,1,0,0,0.0
2,1.689,6.869,mat,0,0,1,0.0
3,1.826,6.446,bio,1,0,0,1.0
4,1.300,6.922,mat,0,0,1,0.0


**Intervalo de Classes (Binnig):** Também conhecido como discretização ...